In [ ]:
# Model 1: https://www.kaggle.com/devm2024/transfer-learning-with-vgg-16-cnn-aug-lb-0-1712
# Model 2: https://www.kaggle.com/vincento/keras-starter-4l-0-1694-lb-icebergchallenge
# Model 3: https://www.kaggle.com/bluevalhalla/fully-convolutional-network-lb-0-193
# Model 4: https://www.kaggle.com/wvadim/keras-tf-lb-0-18

# ResNet50
# InceptionV3
# MobileNet
# DenseNet
# SqueezeNet
# InceptionResNetV2
# Xception

# Simple model: https://www.kaggle.com/devm2024/keras-model-for-beginners-0-210-on-lb-eda-r-d
# https://www.kaggle.com/henokanh/cnn-batchnormalization-0-1646
# https://www.kaggle.com/knowledgegrappler/a-keras-prototype-0-21174-on-pl
# https://www.kaggle.com/cttsai/ensembling-gbms-lb-203/code
# https://www.kaggle.com/yuhaichina/single-model-vgg16-mobilenet-lb-0-1568-with-tf


In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
import keras
import abc

from mpl_toolkits.mplot3d import Axes3D

#Import Keras.
#from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import *
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import rmsprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

#Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [98]:
class Helpers():
    def get_angledata(train, test):
        train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
        train['inc_angle']=train['inc_angle'].fillna(method='pad')
        test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')

        X_angle=train['inc_angle']
        X_test_angle=test['inc_angle']
        
        return X_angle, X_test_angle
    
    def get_imagedata(data):
        X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in data["band_1"]])
        X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in data["band_2"]])
        X_band_3=np.fabs(np.subtract(X_band_1,X_band_2))
        X_band_4=np.maximum(X_band_1,X_band_2)
        X_band_5=np.minimum(X_band_1,X_band_2)
        X_data = np.concatenate([X_band_3[:, :, :, np.newaxis],X_band_4[:, :, :, np.newaxis],X_band_5[:, :, :, np.newaxis]], axis=-1)

        return X_data
    
    def get_generator():
        # Define the image transformations here
        return ImageDataGenerator(horizontal_flip = True,
                                 vertical_flip = True,
                                 width_shift_range = 0.,
                                 height_shift_range = 0.,
                                 channel_shift_range=0,
                                 zoom_range = 0.2,
                                 rotation_range = 10)
    
    # Here is the function that merges our two generators
    # We use the exact same generator with the same random seed for both the y and angle arrays
    def gen_flow_for_two_inputs(X1, X2, y, batch_size = 64):
        gen = Helpers.get_generator()
        genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
        genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
        while True:
                X1i = genX1.next()
                X2i = genX2.next()
                #Assert arrays are equal - this was for peace of mind, but slows down training
                #np.testing.assert_array_equal(X1i[0],X2i[0])
                yield [X1i[0], X2i[1]], X1i[1]
                
    # Here is the function that merges our two generators
    # We use the exact same generator with the same random seed for both the y and angle arrays
    def gen_flow_for_one_input(X1, y, batch_size = 64):
        gen = Helpers.get_generator()
        genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
        return genX1
                

    # Finally create generator
    def get_callbacks(filepath, patience=2):
        es = EarlyStopping('val_loss', patience=10, mode="min")
        msave = ModelCheckpoint(filepath, save_best_only=True)
        reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')
        return [es, msave, reduce_lr_loss]

In [99]:
class ModelBase():
    __metaclass__ = abc.ABCMeta
    
    def __init__(self, ids = None):
        self.batch_size = 64
        self.epochs = 100
        self.ids = ids
        self.predictions = None
        self.K = 5
        
        self.model = self.get_model()
        
    def save_model(self):
        name = self.get_name() + datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d-%H%M%S') + ".h5"
        self.model.save_weights(name)
        
    def create_submission(self, predict):
        submission = pd.DataFrame()
        submission['id']=test['id']
        submission['is_iceberg']=preds
        submission.to_csv("submission-" + self.get_name() + ".csv", float_format='%g', index = False)
    
    @abc.abstractmethod
    def get_model(self):
        pass

    @abc.abstractmethod
    def get_name(self):
        pass
    
        
    #Using K-fold Cross Validation with Data Augmentation.
    def train_predict(self, X_train, X_angle, X_test, target_train):
        folds = list(StratifiedKFold(n_splits=self.K, shuffle=True, random_state=16).split(X_train, target_train))
        y_test_pred_log = 0
        y_train_pred_log=0
        y_valid_pred_log = 0.0*target_train
        for j, (train_idx, test_idx) in enumerate(folds):
            print('\n===================FOLD=',j)
            X_train_cv = X_train[train_idx]
            y_train_cv = target_train[train_idx]
            X_holdout = X_train[test_idx]
            Y_holdout= target_train[test_idx]

            #Angle
            X_angle_cv=X_angle[train_idx]
            X_angle_hold=X_angle[test_idx]

            #define file path and get callbacks
            file_path = "%s_"%j + self.get_name() + ".hdf5"
            callbacks = Helpers.get_callbacks(filepath=file_path, patience=5)
            gen_flow = Helpers.gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
            
            galaxyModel= self.get_model()
            
            input_shape = len(galaxyModel.model.input_shape)
            train_array = [X_train,X_angle]
            train_array_cv = [X_train_cv,X_angle_cv]
            holdout_array = [X_holdout,X_angle_hold]
            test_array = [X_test, X_test_angle]
            
            if input_shape != 2:
                train_array = X_train
                train_array_cv = X_train_cv
                holdout_array = X_holdout
                test_array = X_test
                gen_flow = Helpers.gen_flow_for_one_input(X_train_cv, y_train_cv)
                
            galaxyModel.fit_generator(
                    gen_flow,
                    steps_per_epoch=24,
                    epochs=self.epochs,
                    shuffle=True,
                    verbose=1,
                    validation_data=(holdout_array, Y_holdout),
                    callbacks=callbacks)

            #Getting the Best Model
            galaxyModel.load_weights(filepath=file_path)
            #Getting Training Score
            score = galaxyModel.evaluate(train_array_cv, y_train_cv, verbose=0)
            print('Train loss:', score[0])
            print('Train accuracy:', score[1])
            #Getting Test Score
            score = galaxyModel.evaluate(holdout_array, Y_holdout, verbose=0)
            print('Test loss:', score[0])
            print('Test accuracy:', score[1])

            #Getting validation Score.
            pred_valid=galaxyModel.predict(holdout_array)
            y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

            #Getting Test Scores
            temp_test=galaxyModel.predict(test_array)
            y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

            #Getting Train Scores
            temp_train=galaxyModel.predict(train_array)
            y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

        y_test_pred_log=y_test_pred_log/self.K
        y_train_pred_log=y_train_pred_log/self.K

        print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
        print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
        
#         self.plot_results()
        
        self.predictions = y_test_pred_log 
        return y_test_pred_log
    
#     def plot_results(self):
#         plt.plot(self.history['acc'])
#         plt.plot(self.history['val_acc'])
#         plt.title('model accuracy')
#         plt.ylabel('accuracy')
#         plt.xlabel('epoch')
#         plt.legend(['train', 'test'], loc='upper left')
#         plt.show()
#         # summarize history for loss
#         plt.plot(self.history['loss'])
#         plt.plot(self.history['val_loss'])
#         plt.title('model loss')
#         plt.ylabel('loss')
#         plt.xlabel('epoch')
#         plt.legend(['train', 'test'], loc='upper left')
#         plt.show()
        

In [100]:
class VggModel(ModelBase):
    def get_model(self):
        input_2 = Input(shape=[1], name="angle")
        angle_layer = Dense(1, )(input_2)
        base_model = VGG16(weights='imagenet', include_top=False, 
                     input_shape=X_train.shape[1:], classes=1)
        x = base_model.get_layer('block5_pool').output

        x = GlobalMaxPooling2D()(x)
        merge_one = concatenate([x, angle_layer])
        merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
        merge_one = Dropout(0.3)(merge_one)
        merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
        merge_one = Dropout(0.3)(merge_one)

        predictions = Dense(1, activation='sigmoid')(merge_one)

        model = Model(input=[base_model.input, input_2], output=predictions)

        sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(loss='binary_crossentropy',
                      optimizer=sgd,
                      metrics=['accuracy'])
        return model

    def get_name(self):
        return "vgg16"

In [101]:
class SimpleModel(ModelBase):
    def get_model(self):
        
        
#         input_2 = Input(shape=[1], name="angle")
#         angle_layer = Dense(1, )(input_2)
        
        
        
#         base_model = VGG16(weights='imagenet', include_top=False, 
#                      input_shape=X_train.shape[1:], classes=1)
        
        
        base_model=Sequential()

        # CNN 1
        base_model.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
        base_model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
        base_model.add(Dropout(0.2))

        # CNN 2
        base_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
        base_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        base_model.add(Dropout(0.2))

        # CNN 3
        base_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
        base_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        base_model.add(Dropout(0.3))

        #CNN 4
        base_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
        base_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        base_model.add(Dropout(0.3))

        # You must flatten the data for the dense layers
        base_model.add(Flatten())

        #Dense 1
        base_model.add(Dense(512, activation='relu'))
        base_model.add(Dropout(0.2))

#         #Dense 2
        base_model.add(Dense(256, activation='relu'))
        base_model.add(Dropout(0.2))

        # Output 
        base_model.add(Dense(1, activation="sigmoid"))

        optimizer = Adam(lr=0.001, decay=0.0)
        base_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
        
#         x = base_model.get_layer('block5_pool').output
#         x = base_model.output

# #         x = GlobalMaxPooling2D()(x)
#         merge_one = concatenate([x, angle_layer])
# #         merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
# #         merge_one = Dropout(0.2)(merge_one)
# #         merge_one = Dense(256, activation='relu', name='fc3')(merge_one)
# #         merge_one = Dropout(0.2)(merge_one)

#         predictions = Dense(1, activation='sigmoid')(merge_one)

#         model = Model(input=[base_model.input, input_2], output=predictions)

#         sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
#         model.compile(loss='binary_crossentropy',
#                       optimizer=sgd,
#                       metrics=['accuracy'])
        
        return base_model

    def get_name(self):
        return "simple"

In [6]:
train = pd.read_json("../_RawData/train.json/data/processed/train.json")
test = pd.read_json("../_RawData/test.json/data/processed/test.json")

In [7]:
target_train=train['is_iceberg']
X_angle, X_test_angle = Helpers.get_angledata(train, test)
X_train = Helpers.get_imagedata(train)
X_test = Helpers.get_imagedata(test)
ids = test['id']

In [29]:
model = VggModel(ids)

preds=model.train_predict(X_train, X_angle, X_test, target_train)

C:\Users\Dave\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=[<tf.Tenso...)`



===================FOLD= 0
Epoch 1/2
24/24 [==============================] - 8s 314ms/step - loss: 0.6660 - acc: 0.6314 - val_loss: 0.4915 - val_acc: 0.7460
Epoch 2/2
24/24 [==============================] - 6s 238ms/step - loss: 0.4953 - acc: 0.7404 - val_loss: 0.4786 - val_acc: 0.7347
Train loss: 0.490264658178
Train accuracy: 0.729088639201
Test loss: 0.478633190226
Test accuracy: 0.734744707422

===================FOLD= 1
Epoch 1/2
24/24 [==============================] - 8s 316ms/step - loss: 0.8795 - acc: 0.5184 - val_loss: 0.6784 - val_acc: 0.5605
Epoch 2/2
24/24 [==============================] - 6s 244ms/step - loss: 0.7464 - acc: 0.5283 - val_loss: 0.6360 - val_acc: 0.6567
Train loss: 0.635482364989
Train accuracy: 0.646326276537
Test loss: 0.636023906063
Test accuracy: 0.656679151061

 Train Log Loss Validation=  0.550150008366
 Test Log Loss Validation=  0.557230431963


In [55]:
model = SimpleModel(ids)
print(len(model.model.input_shape))

model.model.summary()

2
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv2d_81_input (InputLayer)    (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
conv2d_81 (Conv2D)              (None, 73, 73, 64)   1792        conv2d_81_input[0][0]            
__________________________________________________________________________________________________
max_pooling2d_78 (MaxPooling2D) (None, 36, 36, 64)   0           conv2d_81[0][0]                  
__________________________________________________________________________________________________
dropout_160 (Dropout)           (None, 36, 36, 64)   0           max_pooling2d_78[0][0]           
__________________________________________________________________________________________________
conv2d_8

C:\Users\Dave\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:66: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=[<tf.Tenso...)`


In [102]:
model = SimpleModel(ids)

preds=model.train_predict(X_train, X_angle, X_test, target_train)


===================FOLD= 0
Epoch 1/100
24/24 [==============================] - 15s 617ms/step - loss: 0.9011 - acc: 0.5481 - val_loss: 0.6251 - val_acc: 0.5186
Epoch 2/100
24/24 [==============================] - 1s 60ms/step - loss: 0.6048 - acc: 0.6180 - val_loss: 0.5572 - val_acc: 0.6708
Epoch 3/100
24/24 [==============================] - 2s 81ms/step - loss: 0.5685 - acc: 0.6825 - val_loss: 0.5303 - val_acc: 0.7081
Epoch 4/100
24/24 [==============================] - 2s 87ms/step - loss: 0.5492 - acc: 0.7057 - val_loss: 0.5112 - val_acc: 0.8075
Epoch 5/100
24/24 [==============================] - 2s 76ms/step - loss: 0.5158 - acc: 0.7511 - val_loss: 0.4513 - val_acc: 0.7795
Epoch 6/100
24/24 [==============================] - 2s 87ms/step - loss: 0.4874 - acc: 0.7665 - val_loss: 0.4319 - val_acc: 0.7950
Epoch 7/100
24/24 [==============================] - 2s 82ms/step - loss: 0.4705 - acc: 0.7848 - val_loss: 0.4091 - val_acc: 0.7981
Epoch 8/100
24/24 [===========================

24/24 [==============================] - 2s 101ms/step - loss: 0.2757 - acc: 0.8811 - val_loss: 0.2698 - val_acc: 0.8944
Epoch 62/100
24/24 [==============================] - 2s 72ms/step - loss: 0.2816 - acc: 0.8872 - val_loss: 0.2708 - val_acc: 0.8913
Epoch 63/100
24/24 [==============================] - 2s 81ms/step - loss: 0.2978 - acc: 0.8676 - val_loss: 0.2683 - val_acc: 0.8882
Epoch 64/100
24/24 [==============================] - 2s 90ms/step - loss: 0.2901 - acc: 0.8735 - val_loss: 0.2671 - val_acc: 0.8882
Epoch 65/100
24/24 [==============================] - 2s 84ms/step - loss: 0.2829 - acc: 0.8800 - val_loss: 0.2662 - val_acc: 0.8789
Epoch 66/100
24/24 [==============================] - 2s 76ms/step - loss: 0.2948 - acc: 0.8487 - val_loss: 0.2709 - val_acc: 0.8851
Epoch 67/100
24/24 [==============================] - 3s 106ms/step - loss: 0.2776 - acc: 0.8846 - val_loss: 0.2613 - val_acc: 0.8975
Epoch 68/100
24/24 [==============================] - 2s 72ms/step - loss: 0.302

23/24 [===========================>..] - ETA: 0s - loss: 0.4137 - acc: 0.8179
Epoch 00023: reducing learning rate to 0.00010000000474974513.
24/24 [==============================] - 3s 139ms/step - loss: 0.4080 - acc: 0.8116 - val_loss: 0.4420 - val_acc: 0.7819
Epoch 24/100
24/24 [==============================] - 2s 79ms/step - loss: 0.4308 - acc: 0.7969 - val_loss: 0.3161 - val_acc: 0.8598
Epoch 25/100
24/24 [==============================] - 2s 92ms/step - loss: 0.3891 - acc: 0.8249 - val_loss: 0.3161 - val_acc: 0.8598
Epoch 26/100
24/24 [==============================] - 2s 94ms/step - loss: 0.3563 - acc: 0.8311 - val_loss: 0.3067 - val_acc: 0.8567
Epoch 27/100
24/24 [==============================] - 2s 79ms/step - loss: 0.4667 - acc: 0.7826 - val_loss: 0.3136 - val_acc: 0.8598
Epoch 28/100
24/24 [==============================] - 2s 84ms/step - loss: 0.3517 - acc: 0.8520 - val_loss: 0.3115 - val_acc: 0.8598
Epoch 29/100
24/24 [==============================] - 2s 94ms/step - loss

KeyboardInterrupt: 

In [19]:
model.create_submission(preds)